In [1]:
import struct
import numpy as np
import os

In [2]:
import matplotlib.pyplot as plt
import healpy as hp
nside = 32
npix = hp.nside2npix(nside)

%matplotlib inline

In [3]:
def convertToSpherical(xyz):
    # col 0: r, col 1: theta, col 2: phi
    sphericalPositons = np.zeros((len(xyz),3))
    xySquared = xyz[:,0]**2 + xyz[:,1]**2
    # r
    sphericalPositons[:,0] = np.sqrt(xySquared+xyz[:,2]**2)
    # theta - many options for this calc
    sphericalPositons[:,1] = np.arctan2(np.sqrt(xySquared), xyz[:,2])
    # phi
    sphericalPositons[:,2] = np.arctan2(xyz[:,1],xyz[:,0])
    return sphericalPositons

In [4]:
# Reading lp-cola unformatted files
def unf_read_file(file, p_list=[], np=6):
    with open(file, mode='rb') as f:
        tot_n = 0
        cnt = 0
        while (True):
            cnt += 1
            r = f.read(4)
            if not r: break
            a1 = struct.unpack('i', r)
            r = f.read(a1[0])
            n = struct.unpack('i', r)
            r = f.read(8)
            a, b = struct.unpack('2i', r)
            r = f.read(b)
            p_list.extend(struct.unpack(str(n[0] * np) + 'f', r))
            r = f.read(4)
            tot_n += n[0]
    f.close()
    return tot_n

In [ ]:
tempArray = []
for i in range(0,1395):
    path = "Data/cone_test_lightcone."+str(i)
    if os.path.isfile(path):
        unf_read_file(path, p_list=tempArray)
        
        positionFormatted = np.reshape(tempArray,(-1,6))[:,0:3]
        sphereCoord = convertToSpherical(positionFormatted)
        
        pixIndicies = hp.ang2pix(nside,sphereCoord[:,1],sphereCoord[:,2])
        
        if not i:
            numcount = np.bincount(pixIndicies, minlength=npix)
        else:
            numcount = np.add(numcount, np.bincount(pixIndicies, minlength=npix) )
        if (i%10==0): 
            print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250


In [ ]:
hp.mollview(numcount,xsize=1600)